#Scrapping the Neighborhood
###Capstone project, Week 3, task1

The task is to get the neighborhood information from wikipedia

In [2]:
#import packages that are commonly used for web scrapping
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'bs4'